In [1]:
# ok. You have an idea of what is PCA - finding dimensions, explaining the data the best way
# how to run PCA?

import numpy as np
from sklearn.decomposition import PCA

# print 3 symbols in fixed notation
np.set_printoptions(precision=3, suppress=True)

X = np.matrix([
    [-1, 0.1, 1],
    [-3, 0.01, 3],
    [-10, 0.02, 10],
    [8, -0.01, -8],
    [6, -0.3, -6],
    [0, -0, 0],
])

pca = PCA(n_components=1)
X_reduced = pca.fit_transform(X)
X_reduced2 = PCA(n_components=2).fit_transform(X)

# note these are just distances from 0!
print("1 component")
print(X_reduced)

print("2 components")
print(X_reduced2)

1 component
[[  1.415]
 [  4.243]
 [ 14.142]
 [-11.313]
 [ -8.487]
 [  0.   ]]
2 components
[[  1.415  -0.119]
 [  4.243  -0.008]
 [ 14.142   0.055]
 [-11.313  -0.104]
 [ -8.487   0.207]
 [  0.     -0.03 ]]


In [2]:
# how to obtain transformation matrix T from 3D to 1D or 2D?
# Obviosly, solve equation!
#     (X - μ) * T = X_reduced
# where (X - μ) is just a centered matrix X

μ = np.mean(X, 0)
X_ = X - μ
# use LSE method
T1 = np.linalg.pinv(X_.T * X_) * X_.T * X_reduced
T2 = np.linalg.pinv(X_.T * X_) * X_.T * X_reduced2
print("Transformation matrix 1D:")
print(T1)
print("Transformed matrix 1D:")
print(((X - μ) * T1))

# exactly the same values as PCA provided, but we can now project other points!
print("Transformation matrix 2D:")
print(T2)
print("Transformed matrix 2D:")
print(((X - μ) * T2))

print("Apply trasformation to NEW data:")
print(np.matrix([[-50, 1, 50]]) * T1)

Transformation matrix 1D:
[[-0.707]
 [ 0.007]
 [ 0.707]]
Transformed matrix 1D:
[[  1.415]
 [  4.243]
 [ 14.142]
 [-11.313]
 [ -8.487]
 [  0.   ]]
Transformation matrix 2D:
[[-0.707 -0.005]
 [ 0.007 -1.   ]
 [ 0.707  0.005]]
Transformed matrix 2D:
[[  1.415  -0.119]
 [  4.243  -0.008]
 [ 14.142   0.055]
 [-11.313  -0.104]
 [ -8.487   0.207]
 [  0.     -0.03 ]]
Apply trasformation to NEW data:
[[70.716]]


In [3]:
from numpy.linalg import svd

U, s, V_T = svd(X - μ)
print("==================== U ======================")
print(U)
print("==================== Σ ======================")
Σ = np.zeros((U.shape[1], V_T.shape[0]), dtype=float)
Σ[:V_T.shape[0], :V_T.shape[0]] = np.diag(s)
print(Σ)
print("==================== V ======================")
print(V_T)
UΣ = U * Σ
print("Look, we got the same matrix 1D!")
print(UΣ[:,:1])
print("Look, we got the same matrix 2D!")
print(UΣ[:,:2])

==================== U ======================
[[-0.069 -0.445  0.874  0.134  0.123 -0.012]
 [-0.207 -0.031 -0.026  0.636 -0.732  0.123]
 [-0.69   0.206 -0.09   0.416  0.547 -0.02 ]
 [ 0.552 -0.389 -0.293  0.572  0.357 -0.066]
 [ 0.414  0.771  0.373  0.271  0.091  0.113]
 [-0.    -0.112 -0.051 -0.063  0.118  0.983]]
==================== Σ ======================
[[20.494  0.     0.   ]
 [ 0.     0.268  0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]]
==================== V ======================
[[ 0.707 -0.007 -0.707]
 [-0.005 -1.     0.005]
 [ 0.707 -0.     0.707]]
Look, we got the same matrix 1D!
[[ -1.415]
 [ -4.243]
 [-14.142]
 [ 11.313]
 [  8.487]
 [ -0.   ]]
Look, we got the same matrix 2D!
[[ -1.415  -0.119]
 [ -4.243  -0.008]
 [-14.142   0.055]
 [ 11.313  -0.104]
 [  8.487   0.207]
 [ -0.     -0.03 ]]
